In [1]:
# parameters
datadir = ""
scan_numbers_string = ""
output = ""
all_scans = "false"
write_tiff = "false"
overwrite = "false"

In [3]:
import os
import re
import glob
import logging
import numpy as np
from nexus2srs import nxs2dat

In [4]:
def replace_scan_number(filename: str, new_number: int) -> str:
    return re.sub(r"\d{3,}.nxs$", str(new_number)+".nxs", filename)

In [5]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [6]:
if not os.path.isdir(datadir):
    raise ValueError("data directory not found")

In [7]:
all_scans = True if all_scans.lower() == "true" else False
write_tiff = True if write_tiff.lower() == "true" else False
overwrite = True if overwrite.lower() == "true" else False

In [ ]:
logger.debug(
    f"""
    Provided arguments:
    Scan Numbers: {scan_numbers_string}
     Output Path: {output}
       All Scans: {all_scans}
      Write TIFF: {write_tiff}
       Overwrite: {overwrite}
    """
    )

In [ ]:
outpath = os.path.join(datadir, output.strip('/'))

logger.debug(f"Created Outpath: {outpath}")

nxs_files = glob.glob(os.path.join(datadir, "*.nxs"))
if len(nxs_files) == 0:
    raise FileNotFoundError(f"No .nxs files found in {datadir}")

base_fname = nxs_files[0]

In [ ]:
if all_scans:
    scans = nxs_files
else:
    if '-' in scan_numbers_string:
        start, stop = scan_numbers_string.split('-')
        scan_numbers = np.arange(int(start), int(stop) + 1)
    else:
        scan_numbers = np.fromstring(scan_numbers_string, dtype=int, sep=',')
    scans = (replace_scan_number(base_fname, n) for n in scan_numbers)

In [ ]:
for scan in scans:
    dat_file = re.sub(".nxs$", ".dat", scan)
    if not os.path.exists(scan):
        logger.warning(f"{scan} does not exist")
        continue
    if os.path.exists(dat_file) and not overwrite:
        logger.warning(f"{dat_file} already exists but overwrite is False")
        continue
    nxs2dat(scan, outpath, write_tiff, overwrite)
    logger.info(f"{scan} successfully converted!")